<div style="font-size:30px" align="center"> <b> Scaping GitHub READMEs </b> </div>

<div style="font-size:18px" align="center"> <b> Brandon Kramer, UVA Biocomplexity Institute, OSS DSPG 2021 </b> </div>

<br>

In this notebook, we have developed a function for scraping GitHub READMEs in order to classify repositories into different types of software projects. 

The pipeline is setup with the following steps: 

1. Loading all of the packages 

2. Calling a function that scrapes the READMEs 

3. Loading the repositories from PostgreSQL as a DataFrame 

4. Cross-referencing the repos against the scraped data 

5. Scraping the repos using multiprocessing 

6. Checking the data that was scraped 

#### Load Packages 

In [2]:
import os 
import glob
import json 
import lxml
import requests 
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup
from datetime import datetime
import psycopg2 as pg
from sqlalchemy import create_engine
import pandas as pd
from ratelimit import limits, sleep_and_retry
import multiprocessing
from multiprocessing.pool import ThreadPool as Pool
print("ready")

ready


#### Call Scraping Function 

To use this function, you first need to set the username and personal acccess token (PAT) that you create on GitHub. While you can run it without this information but having no PAT means you can only make 50 calls an hour compared to about 5,000. It helps if you have several PATs to help in this process, especially since this function takes only a few moments to make 5,000 calls after the multiprocessing is incorporated. In this pipeline, we are calling the usernames and PATs from a table in PostgreSQL. Once the username and PAT are passed into the authetication fields, `requests` will connect to the GitHub repository of all the slugs you feed it. We have designed the function to throw errors for all issues it encounters unless it gets a 404 error (i.e. no README available), as this usually means that the repo or README has been deleted, never existed, and/or is no longer available for some reason. Unfortunately, the way that GitHub's API is setup seems to require two calls to get the README: one to get the JSON with the README location and another to decode the content. We have a `@sleep_and_retry` decorator to deal with this if we setup a `slurm` for each PAT, but using it here in the notebook means that you will just want to wait for the threshold to be hit and then move onto the next PAT. While we plan to continue working on this function to minimize the number of calls, this process will allow us to get some preliminary data for classification in the short-term.  

**NOTE: Before running this cell, you need to set the** `github_pat_index`. **Changing this parameter provides access to 36 different PATs.** 

In [195]:
# set this parameter to a number between 0 and 35 
github_pat_index = 15

# set the environment variables with your username and github personal access token here 
# connect to the database, download data 
connection = pg.connect(host = 'postgis1', database = 'sdad', 
                        user = os.environ.get('db_user'), 
                        password = os.environ.get('db_pwd'))
github_pats = '''SELECT * FROM gh_2007_2020.pats'''
github_pats = pd.read_sql_query(github_pats, con=connection)
github_username = github_pats.login[github_pat_index]
github_token = github_pats.token[github_pat_index]

# os.environ['GITHUB_USERNAME'] = ''
# os.environ['GITHUB_TOKEN'] = ''
# github_username = os.environ.get("GITHUB_USERNAME")
# github_token = os.environ.get("GITHUB_TOKEN")

# can only make 2500 calls per hour 
# because the function calls twice each time 
@sleep_and_retry
@limits(calls=2500, period=3600)
def scrape_readmes(slug):
    
    while True:
        try: 
            # define url based on the slug 
            url = f'https://api.github.com/repos/{slug}/readme'
            response = requests.get(url, auth=(github_username, github_token))
            response_code = response.status_code
            
            if response_code == 404: 
                print(f"404 error on {slug}")
                readme_string = "404 ERROR - NO README"
                now = datetime.now()
                current_time = now.strftime("%Y-%m-%d %H:%M:%S")
                return slug, readme_string, current_time, "Done"
            
            # can i build in a response_code == 403 continue onto the next PAT in the PAT for loop 
            
        except KeyError:
            print("Key error for: " + slug, flush=True)
            break
        
        except requests.exceptions.HTTPError as http_error:
            print ("HTTP Error:", http_error)
            raise SystemExit(http_error)
            break
            
        except requests.exceptions.ConnectionError as connection_error:
            print ("Error Connecting:", connection_error)
            raise SystemExit(connection_error)
            break 
        
        except requests.exceptions.TooManyRedirects as toomany_requests:
            print ("Too Many Requests:", toomany_requests)
            raise SystemExit(toomany_requests)
            break
                
        except requests.exceptions.Timeout as timeout_error:
            print ("Timeout Error:", timeout_error)
            raise SystemExit(timeout_error)
            break
        
        except requests.exceptions.RequestException as request_exception_error:
                print ("Oops, Some Other Error:", request_exception_error)
                raise SystemExit(request_exception_error)
                break 
            
        html_content = response.content
        soup = BeautifulSoup(html_content, 'html.parser')
        site_json=json.loads(soup.text)
        readme_link = site_json['download_url']
        
        while True:
            try: 
                readme_response = requests.get(readme_link, auth=(github_username, github_token))
                readme_response_code = readme_response.status_code
                
            except requests.exceptions.HTTPError as http_error:
                print ("HTTP Error:", http_error)
                raise SystemExit(http_error)
                break
            
            except requests.exceptions.ConnectionError as connection_error:
                print ("Error Connecting:", connection_error)
                raise SystemExit(connection_error)
                break
            
            except requests.exceptions.TooManyRedirects as toomany_requests:
                print ("Too Many Requests:", toomany_requests)
                raise SystemExit(toomany_requests)
                break 
        
            except requests.exceptions.Timeout as timeout_error:
                print ("Timeout Error:", timeout_error)
                raise SystemExit(timeout_error)
                break
        
            except requests.exceptions.RequestException as request_exception_error:
                print ("Oops, Some Other Error:", request_exception_error)
                raise SystemExit(request_exception_error)
                break  
    
            # pull the content out of the readme 
            readme_content = readme_response.content
            readme_soup = BeautifulSoup(readme_content, 'html.parser')
            readme_string = str(readme_soup)
    
            #give us the the timing and status 
            now = datetime.now()
            current_time = now.strftime("%Y-%m-%d %H:%M:%S")
            #print(readme_string)
            return slug, readme_string, current_time, "Done"
    
if __name__ == "__main__":
    print("Started scraping")
    scrape_readmes(slug = 'cjbd/src')
    print("Finished scraping")

Started scraping
404 error on cjbd/src
Finished scraping


#### Ingesting the Repo Slugs from the Database 

Here, we ingested the repository data from PostgreSQL. To pull from different subsets of the data, you can change the range of the commits clause in the `SQL` code. Even if you pull from the same range as someone else already has, the next step of the pipeline is cross-referencing which READMEs have already been scraped and then removing from slugs from the dataset. I have also added a clause to remove all of the repos with an 'Init' status, as their commits data have not yet been scraped and they seem to missing in some systematic way. For now, we are just ignoring them to deal with the majority of valid repos.

In [174]:
# connect to the database, download data 
connection = pg.connect(host = 'postgis1', database = 'sdad', 
                        user = os.environ.get('db_user'), 
                        password = os.environ.get('db_pwd'))

raw_slug_data = '''SELECT * FROM gh_2007_2020.repos_ranked where commits < 112 AND commits > 100 AND status != 'Init' '''

# convert to a dataframe, show how many missing we have (none)
raw_slug_data = pd.read_sql_query(raw_slug_data, con=connection)
raw_slug_data.head()

,id,spdx,slug,createdat,description,primarylanguage,branch,commits,asof,status
0,MDEwOlJlcG9zaXRvcnkxODAzNjIyMw==,MIT,ChristianKniep/docker-elk,2014-03-23 15:25:46,Dockerfile creating ELK services (Elasticsearc...,Shell,MDM6UmVmMTgwMzYyMjM6cmVmcy9oZWFkcy9tYXN0ZXI=,111,2021-01-03 14:38:07,Done
1,MDEwOlJlcG9zaXRvcnkxMTg2NjcwMjE=,GPL-2.0,GregoryGhost/gasoline,2018-01-23 20:42:12,Sample interop application WPF on C# + Library...,F#,MDM6UmVmMTE4NjY3MDIxOnJlZnMvaGVhZHMvbWFzdGVy,111,2021-01-03 21:04:20,Done
2,MDEwOlJlcG9zaXRvcnkxNjk4MDE3OTU=,GPL-3.0,sebastianoverdolini/swordx,2019-02-08 21:32:32,Unix/Linux system application that collects al...,C,MDM6UmVmMTY5ODAxNzk1OnJlZnMvaGVhZHMvbWFzdGVy,111,2021-01-03 14:14:15,Done
3,MDEwOlJlcG9zaXRvcnkxNDI1NTgxNDI=,BSD-3-Clause,SodiumFRP/purescript-sodium,2018-07-27 09:37:34,None,PureScript,MDM6UmVmMTQyNTU4MTQyOnJlZnMvaGVhZHMvbWFzdGVy,111,2021-01-03 16:55:57,Done
4,MDEwOlJlcG9zaXRvcnkxMDAzMzExODY=,MIT,emilio93/digitales-ii,2017-08-15 02:45:20,curso de digitales ii,Verilog,MDM6UmVmMTAwMzMxMTg2OnJlZnMvaGVhZHMvbWFzdGVy,111,2021-01-04 04:57:00,Done


#### Checking the Counts 

Before running the function, you want to compare the original table that you just pulled from the database and the local CSV files that are keeping track of the already downloaded data. The first cell below gives you the count for the original table and the next cell pulls in all of the CSVs, concatenates them, and then gives the count of how many more slugs need to be scraped. 

In [206]:
raw_slug_data['slug'].count()

64221

In [205]:
os.chdir('/project/class/bii_sdad_dspg/ncses_oss_2021/requests_scrape/')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_csv = combined_csv[combined_csv['status'] == 'Done']
already_scraped_slugs = combined_csv['slug'].tolist()
filtered_slugs = ~raw_slug_data.slug.isin(already_scraped_slugs)
filtered_slugs = raw_slug_data[filtered_slugs]
filtered_slugs.head()
filtered_slugs['slug'].count()

57188

#### Scraping the READMEs 

This chunk of code does a few things. First, it sets the `batch_name` to both keep track of which batch of data we are collecting and to name the output files with the appropriate name. This MUST be done before you run the code chunk. Second, the code sets up multiprocessing to draw from multiple cores. Third, the code converts the DataFrame into a list to feed the slugs into the for loop. Lastly, we feed all the slugs to the function as a for loop and it downloads the data into our project folder. 

**NOTE: Before running this cell, set the** `batch_name` **variable. Changing this variable will tell us which download batch the data was collected during and then save the CSV with that batch name.** 

In [ ]:
# need to change this for each batch or you will save over what you had  
batch_name = 'oss_readme_batch1_26' 

# sets the number of cores so that it can draw from multiprocessors 
# there must be 1 core subtracted so that the notebook can run too 
cores_available = multiprocessing.cpu_count() - 1
pool = Pool(cores_available)

# convert the dataframe into a list for the subsequent for loop 
raw_slugs = filtered_slugs["slug"].tolist()
slugs = []
for s in raw_slugs:
    slugs.append(s.strip())
    
# now we will feed in all of the remaining slugs 
slug_log = []
readme_log = []
asof_log = []
status_log = []
for result in pool.imap_unordered(scrape_readmes, slugs):
    slug_log.append(result[0])
    readme_log.append(result[1])
    asof_log.append(result[2])
    status_log.append(result[3])
    final_log = pd.DataFrame({'slug': slug_log, "readme_text": readme_log, 'batch': batch_name, 'as_of': asof_log, 'status': status_log}, 
                              columns=["slug", "readme_text", "batch", "as_of", "status"])
    final_log.to_csv('/project/class/bii_sdad_dspg/ncses_oss_2021/requests_scrape/'+batch_name+'.csv', sep=',', encoding='utf-8', index=False)
print("Finished scraping", len(final_log), "of", len(slugs), "records")

#### Common/Known Errors 

Here are some common/known errors that I have identified: 

1. `"MarkupResemblesLocatorWarning: "{slug}" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.` 

    Solution: This will just throw a warning and does not seemt to affect subsequent runs of the dataset. You can ignore this. 
    
2. `KeyError: 'download_url'`

    Solution: It seems like this error can mean one of two things. The first is insignificant and 

#### Examining the Dataset 

This code just pulls in all of the downloaded data as a DataFrame and sees how many entries we have downloaded already.

In [207]:
os.chdir('/project/class/bii_sdad_dspg/ncses_oss_2021/requests_scrape/')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_csv = combined_csv[combined_csv['status'] == 'Done']
combined_csv = combined_csv.sort_values("batch")
combined_csv#.count()
#combined_csv.to_csv('/project/class/bii_sdad_dspg/ncses_oss_2021/requests_scrape/oss_readme_aggregated/oss_readme_data_061521.csv', sep=',', encoding='utf-8', index=False)

,slug,readme_text,batch,as_of,status
510,huangyingw/kubernetes-kubernetes,# Kubernetes\n\n[![GoDoc Widget]][GoDoc] [![CI...,oss_readme_batch1_1,6/11/21 17:21,Done
463,dmanolescu/moodle,.-..-.\n __...,oss_readme_batch1_1,6/11/21 17:21,Done
464,manuelantoniomelendezandrade/moodle,.-..-.\n __...,oss_readme_batch1_1,6/11/21 17:21,Done
465,erkrishna9/odoo,[![Build Status](http://runbot.odoo.com/runbot...,oss_readme_batch1_1,6/11/21 17:21,Done
466,dugasfrederick/Moodle,.-..-.\n __...,oss_readme_batch1_1,6/11/21 17:21,Done
...,...,...,...,...,...
674,justinhj/elm-architecture-tutorial,# Elm\n\nElm is a programming language that co...,oss_readme_batch1_9,2021-06-14 15:06:32,Done
673,Caaraya/Jazz,# Jazz\nA code editor written in C++ with Gtk\...,oss_readme_batch1_9,2021-06-14 15:06:32,Done
672,crykn/jpapi,# jpapi\njpapi ist ein Java Wrapper für die [P...,oss_readme_batch1_9,2021-06-14 15:06:32,Done
685,fo-am/crapapppro,# crapapppro\nHelping to reduce fertiliser use...,oss_readme_batch1_9,2021-06-14 15:06:32,Done


^ That tells you the number of entries we have downloaded. 

#### Development Space 

Below, are just some snippets of code that might be useful when tweaking the existing code. 

In [123]:
os.chdir('/project/class/bii_sdad_dspg/ncses_oss_2021/requests_scrape/')
check = pd.read_csv('oss_readme_batch1_8.csv')
check

,slug,readme_text,batch,as_of,status
0,shahabsaf1/copy,# [InfernalTG](https://telegram.me/TeleInferna...,oss_readme_batch1_8,2021-06-14 12:20:52,Done
1,stephan0992/week-1,# Jekyll Now\n\n**Jekyll** is a static site ge...,oss_readme_batch1_8,2021-06-14 12:20:52,Done
2,SynapseProject/handlers.ActiveDirectory.net,# handlers.ActiveDirectory.net\nActive Directo...,oss_readme_batch1_8,2021-06-14 12:20:52,Done
3,mraggi/discreture,[![Build Status](https://travis-ci.org/mraggi/...,oss_readme_batch1_8,2021-06-14 12:20:52,Done
4,karlstroetmann/Lineare-Algebra,Lineare-Algebra\n===============\n\nIn diesem ...,oss_readme_batch1_8,2021-06-14 12:20:52,Done
...,...,...,...,...,...
2174,pkalro/project8,# angular-seed â€” the seed for AngularJS apps...,oss_readme_batch1_8,2021-06-14 12:35:51,Done
2175,Tizzio/WifiTransfer,# WifiTransfer\nDirect wifi file transfer betw...,oss_readme_batch1_8,2021-06-14 12:35:51,Done
2176,PAPARA-ZZ-I/PAPARA-ZZ-I,# PAPARA(ZZ)I\nCopyright 2015-2017 Yann Marcon...,oss_readme_batch1_8,2021-06-14 12:35:51,Done
2177,Capital-T-Industries/docker-elk,"# The ELK stack (Elasticsearch, Logstash, Kiba...",oss_readme_batch1_8,2021-06-14 12:35:51,Done


In [ ]:
# Copy the function from above and make tweaks in this code chunk

In [36]:
test_slugs = ["brandonleekramer/diversity", 
              'cjbd/src'
              "uva-bi-sdad/oss-2020", 
              "facebook/react", 
              "RichardLitt/standard-readme",            
]

# need to change this for each batch or you will save over what you had  
batch_name_test = "nothing"
cores_available = multiprocessing.cpu_count() - 1
pool = Pool(cores_available)

slug_log = []
readme_log = []
asof_log = []
status_log = []
for result in pool.imap_unordered(scrape_readmes, test_slugs):
    slug_log.append(result[0])
    readme_log.append(result[1])
    asof_log.append(result[2])
    status_log.append(result[3])
    final_log = pd.DataFrame({'slug': slug_log, "readme_text": readme_log, 'batch': batch_name_test, 'as_of': asof_log, 'status': status_log}, 
                              columns=["slug", "readme_text", "batch", "as_of", "status"])
print("Finished scraping", len(final_log), "of", len(test_slugs), "records")

404 error on cjbd/srcuva-bi-sdad/oss-2020
Finished scraping 4 of 4 records
